In [12]:
import os
import wandb
os.environ["WANDB_API_KEY"]="fb22019a857cb5bd0b79ccd93abf9de65ea6a2ee"
os.environ["WANDB_ENTITY"]="Mathiponds"
os.environ["WANDB_PROJECT"]="question_answering"
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"

## Notes 
- Pour l'instant, on test sur le validation set. Il faut regarder si le validation set est utiliser durant le training ou s'il est juste la pour l'évaluation. 
- Pour la traduction, le fait de traduire va peut-être faire que la variable `answer_start` va être déplacée mais je pense que ça n'a pas d'impact car on ne l'utilise pas à la validation
- Le model anglais est train sur 4x plus de samples : est ce qu'on veut train sur plus d'epoch sur le francais 
- La traduction de la réponse est pas fait en même temps que celle de la question

In [3]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer
from datasets.dataset_dict import DatasetDict
from datasets import Dataset
import pandas as pd
import numpy as np

True
2
0


'NVIDIA TITAN Xp'

In [31]:
model_name = {}
model_name['fr'] = 'camembert-base'
model_name['en'] = 'roberta-base'
dataset_name = {}
dataset_name['fr'] = 'fquad'
dataset_name['en'] = 'squad'
data_path = '/data/desponds/data/Question_answering/'

In [5]:
dataset_en = load_dataset(dataset_name['en'])
dataset_fr = load_dataset('json', data_files={'train': data_path+'/fquad/train.json',
                                              'valid': data_path+'/fquad/valid.json'},  field='data')

tokenizer = {}
tokenizer['fr'] = AutoTokenizer.from_pretrained(model_name['fr'])
tokenizer['en'] = AutoTokenizer.from_pretrained(model_name['en'])

Found cached dataset squad (/home/desponds/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

Found cached dataset json (/home/desponds/.cache/huggingface/datasets/json/default-8a2295f7839d92a5/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
#Function taken from https://huggingface.co/docs/transformers/tasks/question_answering
max_length = 384
stride = 128
def preprocess_training_examples(examples, language):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer[language](
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_offsets_mapping=True,
        return_overflowing_tokens=True,
        padding="max_length",
    )
    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []
    
    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)
        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1
        
        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)
            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

def preprocess_validation_examples(examples, language):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer[language](
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [8]:
tokenized_en = DatasetDict()
tokenized_en['train'] = dataset_en['train'].map(lambda examples : preprocess_training_examples(examples, 'en'), 
                                                batched=True, remove_columns=dataset_en["train"].column_names)
tokenized_en['validation'] = dataset_en['validation'].map(lambda examples : preprocess_validation_examples(examples, 'en'), 
                                                batched=True, remove_columns=dataset_en["train"].column_names)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [9]:
tokenized_fr = DatasetDict()
tokenized_fr['train'] = dataset_fr['train'].map(lambda examples : preprocess_training_examples(examples, 'fr'), 
                                                batched=True, remove_columns=dataset_fr["train"].column_names)
tokenized_fr['validation'] = dataset_fr['valid'].map(lambda examples : preprocess_validation_examples(examples, 'fr'), 
                                                batched=True, remove_columns=dataset_fr["train"].column_names)

Map:   0%|          | 0/20731 [00:00<?, ? examples/s]

Map:   0%|          | 0/3188 [00:00<?, ? examples/s]

In [32]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

# Load the models for both language
model = {}
model['fr'] = AutoModelForQuestionAnswering.from_pretrained(model_name['fr'])
model['en'] = AutoModelForQuestionAnswering.from_pretrained(model_name['en'])

#Set the training arguments
def get_training_args(language):
    return TrainingArguments(
        output_dir=data_path+ 'trainer_'+language,
        overwrite_output_dir = True,
        evaluation_strategy="no",
        learning_rate=2e-5,
        num_train_epochs=3,
        weight_decay=0.01,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        save_strategy = 'epoch',
        logging_strategy = 'no',
        report_to=None
    )
def get_trainer(dataset, language):
    return Trainer(
        model=model[language],
        args=training_args[language],
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"],
        tokenizer=tokenizer[language])

training_args = {}
training_args['fr'] = get_training_args('fr')
training_args['en'] = get_training_args('en')
trainer = {}
trainer['fr'] = get_trainer(tokenized_fr, 'fr')
trainer['en'] = get_trainer(tokenized_en, 'en')

loading configuration file config.json from cache at /home/desponds/.cache/huggingface/hub/models--camembert-base/snapshots/3f452b6e5a89b0e6c828c9bba2642bc577086eae/config.json
Model config CamembertConfig {
  "_name_or_path": "camembert-base",
  "architectures": [
    "CamembertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 5,
  "classifier_dropout": null,
  "eos_token_id": 6,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "camembert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 32005
}

loading weights file pytorch_model.bin from cache at /home/desponds/.cache/huggingface/hub/models--camembert-base/snapsh

In [35]:
trainer['fr'].train(data_path+'/trainer_fr/checkpoint-2010')
trainer['en'].train(data_path+'/trainer_en/checkpoint-8304')

Loading model from /data/desponds/data/Question_answering//trainer_fr/checkpoint-2010.
***** Running training *****
  Num examples = 21429
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2010
  Number of trainable parameters = 110032898
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 3
  Continuing training from global step 2010
  Will skip the first 3 epochs then the first 0 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


0it [00:00, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




Step,Training Loss


Loading model from /data/desponds/data/Question_answering//trainer_en/checkpoint-8304.
***** Running training *****
  Num examples = 88567
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 8304
  Number of trainable parameters = 124056578
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 3
  Continuing training from global step 8304
  Will skip the first 3 epochs then the first 0 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


0it [00:00, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




Step,Training Loss


TrainOutput(global_step=8304, training_loss=0.0, metrics={'train_runtime': 0.093, 'train_samples_per_second': 2857133.836, 'train_steps_per_second': 89294.505, 'total_flos': 5.207010717113395e+16, 'train_loss': 0.0, 'epoch': 3.0})

In [15]:
trainer['fr'].save_model('/data/desponds/data/Question_answering/models/model_fr.pt')
trainer['en'].save_model('/data/desponds/data/Question_answering/models/model_en.pt')

Saving model checkpoint to /data/desponds/data/Question_answering/models/model_en.pt
Configuration saved in /data/desponds/data/Question_answering/models/model_en.pt/config.json
Model weights saved in /data/desponds/data/Question_answering/models/model_en.pt/pytorch_model.bin
tokenizer config file saved in /data/desponds/data/Question_answering/models/model_en.pt/tokenizer_config.json
Special tokens file saved in /data/desponds/data/Question_answering/models/model_en.pt/special_tokens_map.json


In [20]:
model_trained = {}
model_trained['fr'] = AutoModelForQuestionAnswering.from_pretrained('/data/desponds/data/Question_answering/models/model_fr.pt')
model_trained['en'] = AutoModelForQuestionAnswering.from_pretrained('/data/desponds/data/Question_answering/models/model_en.pt')

loading configuration file /data/desponds/data/Question_answering/models/model_fr.pt/config.json
Model config CamembertConfig {
  "_name_or_path": "/data/desponds/data/Question_answering/models/model_fr.pt",
  "architectures": [
    "CamembertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 5,
  "classifier_dropout": null,
  "eos_token_id": 6,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "camembert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 32005
}

loading weights file /data/desponds/data/Question_answering/models/model_fr.pt/pytorch_model.bin
All model checkpoint we

## Translation

In [7]:
from helper import translate_fr_en
import pickle

def translate_fr_en_qa(example):
    example['context'] = translate_fr_en(example['context'])
    example['question'] = translate_fr_en(example['question'])
    example['answers']['text'] = [translate_fr_en(example['answers']['text'][0])]
    # If we can find the traduction in the text directly we update the ànswer_start` var
    idx = example['context'].find(example['answers']['text'][0])
    example['answers']['answer_start'] = [idx] if idx != -1 else example['answers']['answer_start']
    return example

# Translate the test split of the french dataset
translated_fr_en = dataset_fr['valid'].map(translate_fr_en_qa)

with open('/data/desponds/data/Question_answering/translated_dataset.pickle', 'wb') as handle:
    pickle.dump(translated_fr_en, handle)

/data/desponds/anaconda3/lib/python3.9/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/3188 [00:00<?, ? examples/s]

Les deux tableaux sont certes décrits par des documents contemporains à leur création mais ceux-ci ne le font qu'indirectement car ils concernent principalement La Vierge aux rochers. Aussi demeurent-ils objets de spéculations pour les chercheurs quant à leur statut de première ou seconde version de l'œuvre, leur création, leur attribution, leur datation, leur disposition exacte sur le retable et les raisons qui ont poussé à leurs modifications au cours du temps — notamment pour ce qui concerne la couleur du fond.


/data/desponds/anaconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Que concerne principalement les documents ?
La Vierge aux rochers
Les deux tableaux sont certes décrits par des documents contemporains à leur création mais ceux-ci ne le font qu'indirectement car ils concernent principalement La Vierge aux rochers. Aussi demeurent-ils objets de spéculations pour les chercheurs quant à leur statut de première ou seconde version de l'œuvre, leur création, leur attribution, leur datation, leur disposition exacte sur le retable et les raisons qui ont poussé à leurs modifications au cours du temps — notamment pour ce qui concerne la couleur du fond.
Par quoi sont décrit les deux tableaux ?
documents contemporains
Les deux tableaux sont certes décrits par des documents contemporains à leur création mais ceux-ci ne le font qu'indirectement car ils concernent principalement La Vierge aux rochers. Aussi demeurent-ils objets de spéculations pour les chercheurs quant à leur statut de première ou seconde version de l'œuvre, leur création, leur attribution, leur d

In [1]:
import pickle
with open('/data/desponds/data/Question_answering/translated_dataset.pickle', 'rb') as handle:
    translated_fr_en = pickle.load(handle)

In [2]:
translated_fr_en[0]

{'id': 'a524504f-0816-4f58-9f2d-27f82a85c73d',
 'title': 'anges-musiciens-(national-gallery)',
 'context': 'The two paintings are certainly described by contemporary documents at their creation, but they do so only indirectly because they relate mainly to The Virgin to the Rocks. Therefore, they remain subject to speculation for the researchers as to their status as the first or second version of the work, their creation, their attribution, their dating, their exact disposition on the retable and the reasons which led to their modifications over time — especially as regards the colour of the background.',
 'question': 'What is the main concern of the documents?',
 'answers': {'answer_start': [161], 'text': ['The Virgin with the Rocks']}}

In [17]:
#Recompute the tokens of the translated version
tokenized_translated_fr_en = translated_fr_en.map(lambda examples : preprocess_validation_examples(examples, 'en'), 
                                                batched=True, remove_columns=dataset_en["train"].column_names)

Map:   0%|          | 0/3188 [00:00<?, ? examples/s]

## Evaluation 

In [27]:
from tqdm.auto import tqdm
import evaluate
from collections import defaultdict

metric = evaluate.load("squad")
n_best =20
max_answer_length = 30

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [38]:
# ON hugging face they manage to get {"f1": 88.3, "exact_match": 78.0}
predictions_fr, _, _ = trainer['fr'].predict(tokenized_fr['validation'])
start_logits_fr, end_logits_fr = predictions_fr
compute_metrics(start_logits_fr, end_logits_fr, tokenized_fr['validation'], dataset_fr["valid"])

The following columns in the test set don't have a corresponding argument in `CamembertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id. If offset_mapping, example_id are not expected by `CamembertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3436
  Batch size = 32


  0%|          | 0/3188 [00:00<?, ?it/s]

{'exact_match': 45.38895859473024, 'f1': 73.3688528648772}

In [19]:
predictions_en, _, _ = trainer['en'].predict(tokenized_en['validation'])
start_logits_en, end_logits_en = predictions_en
compute_metrics(start_logits_en, end_logits_en, tokenized_en['validation'], dataset_en["validation"])

The following columns in the test set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id. If offset_mapping, example_id are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 10790
  Batch size = 32


  0%|          | 0/10570 [00:00<?, ?it/s]

{'exact_match': 85.76158940397352, 'f1': 92.13417617957116}

In [28]:
predictions_fr_en, _, _ = trainer['en'].predict(tokenized_translated_fr_en)
start_logits_fr_en, end_logits_fr_en = predictions_fr_en
compute_metrics(start_logits_fr_en, end_logits_fr_en, tokenized_translated_fr_en, translated_fr_en)

The following columns in the test set don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `RobertaForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3391
  Batch size = 32


  0%|          | 0/3188 [00:00<?, ?it/s]

{'exact_match': 41.060225846925974, 'f1': 65.32442339320636}

## Results

In [29]:
import pandas as pd
data = {
    'task' : ['Question_answering', 'Question_answering', 'Question_answering'],
    'dataset' : ['fquad', 'squad', 'fquad'],
    'translated' : ['no', 'no', 'yes'],
    'model'   : ['CamemBERT', 'Roberta', 'Roberta'],
    'f1_score' : [73.368852, 92.134176, 65.32442339320636 ],
    'exact_match' : [45.388958, 85.761589, 41.060225846925974]
}
results = pd.DataFrame(data)
results

,task,dataset,translated,model,f1_score,exact_match
0,Question_answering,fquad,no,CamemBERT,73.368852,45.388958
1,Question_answering,squad,no,Roberta,92.134176,85.761589
2,Question_answering,fquad,yes,Roberta,65.324423,41.060226
